## Analysis sparks multiple images

In [ ]:
from os import listdir
from os.path import isfile, join
from sparks_image import *
from sparks_analysis import *
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

x_calibracion = 4.5
path = 'C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/Imagenes_confocal/Rata/C071112/'
files = [".".join(f.split(".")[:-1]) for f in listdir(path) if isfile(join(path, f))]
for photo_name in files:
    image = cv2.imread(path + photo_name + ".tif")    # Read image
    plt.imshow(image, cmap='gray')
    cont = input('Continue?')
    while 's' in cont.lower():
        list_img = image_process (image,path)
        list_img_col = list_img [0]
        list_img_row = list_img [1]
        out_sparks = analysis_process (list_img_col, list_img_row)
        out_sparks
        if 'n' in cont.lower():
            continue

## Sparks analysis single image

In [4]:
from sparks_image import *
from sparks_analysis import *
import cv2

path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Sp/Imágenes confocal/Rata/C071112/'
photo_name = 'c2ac000'
image = cv2.imread(path + photo_name + ".tif")    # Read image
list_img = image_process (image,path)
list_img_col = list_img [0]
list_img_row = list_img [1]
out_sparks = analysis_process (list_img_col, list_img_row)

In [6]:
img_mean = (np.mean(image))
img_max = (np.max(image))
img_min = (np.min(image))
img_std = (cv2.meanStdDev(image))
print(img_mean,img_max, img_min, img_std)

15.477653503417969 93 2 (array([[15.4776535],
       [15.4776535],
       [15.4776535]]), array([[11.33300795],
       [11.33300795],
       [11.33300795]]))


In [8]:
out_sparks

,tiempo_maximo,intensidad_maxima,tiempo_minimo,intensidad_minima,tiempo_valle,intensidad_valle,amplitud,TTP,TTP50,FDHM,tau,(ΔF/F0)/ΔTmax,fullDuration,fullWidth,FWHM
0,4.0,130.384615,0,68.923077,11,78.307692,0.891741,4.0,2.232491,4.834375,15.699474,0.222935,11,10,3.821075
1,4.0,169.500000,1,55.400000,8,120.400000,2.059567,3.0,1.606464,3.867328,12.504987,0.686522,7,9,3.649862


In [ ]:
out_sparks.to_csv(path+photo_name+'_sparks.csv', sep='\t')

In [5]:
# LinAlgError('Incompatible dimensions')
list_img[0]
# lstsq(a, b, rcond)
# m2, n_rhs = b.shape[-2:]
# if m != m2:

[[68.92307692307692,
  86.53846153846153,
  89.07692307692308,
  118.0,
  130.3846153846154,
  107.46153846153847,
  117.3846153846154,
  112.53846153846153,
  91.76923076923077,
  95.53846153846153,
  88.6923076923077,
  78.3076923076923,
  78.38461538461539],
 [73.4,
  55.4,
  98.0,
  148.0,
  169.5,
  161.2,
  163.7,
  143.6,
  120.40000000000002,
  126.09999999999998]]

from __future__ import print_function
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

img_bc = interactive(f, brightness=100, contrast=100)   #display brightness and contrast
display (img_bc)

im = rotation (imCrop, 90)    # rotate image
display_image ('Image' , im)    # Display cropped image